In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [2]:
#Import Lib requirement 
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

import glob
import random
import shutil

In [3]:
class_dict = {
    'cam_nguoc_chieu': 1,
    'cam_dung_va_do': 2,
    'cam_re': 3,
    'gioi_han_toc_do': 4,
    'cam_con_lai' : 5,
    'nguy_hiem' : 6,
    'hieu_lenh' : 7
}

In [4]:
df_train = pd.read_csv("/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/annotations/csv_data/train_is.csv")
df_train.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,area
0,3.png,1622,626,cam_dung_va_do,880,333,899,351,342
1,5.png,1622,626,cam_re,1069,355,1152,438,6889
2,5.png,1622,626,cam_dung_va_do,768,480,777,487,63
3,6.png,1622,626,nguy_hiem,781,337,798,352,255
4,16.png,1622,626,cam_dung_va_do,733,352,740,360,56


In [ ]:
len(df_train)

7636

In [5]:
df_val = pd.read_csv("/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/annotations/csv_data/validate_is.csv")
df_val.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,61.png,1622,626,cam_re,1156,291,1202,329
1,245.png,1622,626,cam_dung_va_do,842,369,850,378
2,245.png,1622,626,cam_con_lai,860,373,868,381
3,261.png,1622,626,cam_re,1156,64,1230,127
4,263.png,1622,626,nguy_hiem,1176,402,1202,422


In [ ]:
len(df_val)

776

In [6]:
a = df_train['filename']
train_image_id_list = list(set(a))

In [ ]:
len(train_image_id_list)

3934

In [7]:
b = df_val['filename']
val_image_id_list = list(set(b))

In [ ]:
len(val_image_id_list)

412

In [8]:
df_tfrecord = pd.read_csv("/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/annotations/process_csv/df_tfreocord.csv")
df_tfrecord.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,3.png,1622,626,cam_dung_va_do,880,333,899,351
1,5.png,1622,626,cam_re,1069,355,1152,438
2,5.png,1622,626,cam_dung_va_do,768,480,777,487
3,6.png,1622,626,nguy_hiem,781,337,798,352
4,16.png,1622,626,cam_dung_va_do,733,352,740,360


In [ ]:
len(df_tfrecord)

11000

In [9]:
a = df_tfrecord['filename']
image_id_list = list(set(a))

In [ ]:
len(image_id_list)

4500

In [10]:
validate_id = [x.strip() for x in val_image_id_list]

In [11]:
len(validate_id)

412

In [12]:
validate_id

['5799.png',
 '5789.png',
 '1802.png',
 '5901.png',
 '9663.png',
 '1341.png',
 '5603.png',
 '12387.png',
 '851.png',
 '8540.png',
 '8688.png',
 '3360.png',
 '8206.png',
 '4103.png',
 '7747.png',
 '10966.png',
 '7288.png',
 '4984.png',
 '8460.png',
 '3387.png',
 '7619.png',
 '4176.png',
 '8618.png',
 '12287.png',
 '3593.png',
 '11809.png',
 '8127.png',
 '12203.png',
 '5206.png',
 '4825.png',
 '4221.png',
 '4720.png',
 '6988.png',
 '752.png',
 '7607.png',
 '5389.png',
 '12521.png',
 '10196.png',
 '7449.png',
 '6161.png',
 '10656.png',
 '11582.png',
 '5655.png',
 '5005.png',
 '11088.png',
 '4948.png',
 '7707.png',
 '629.png',
 '263.png',
 '342.png',
 '3054.png',
 '10340.png',
 '3096.png',
 '9918.png',
 '9218.png',
 '5422.png',
 '8131.png',
 '7214.png',
 '4743.png',
 '4035.png',
 '1878.png',
 '365.png',
 '5375.png',
 '5640.png',
 '6253.png',
 '3928.png',
 '611.png',
 '9312.png',
 '704.png',
 '557.png',
 '12474.png',
 '854.png',
 '8391.png',
 '12405.png',
 '7349.png',
 '4517.png',
 '4600.pn

In [13]:
centerX = []
centerY = []
image_id = []
class_id = []
widthnew = []
heightnew = []

for key,row in df_val.iterrows():
    widthnew.append((row["xmax"]-row["xmin"])/1622)
    heightnew.append((row["ymax"]-row["ymin"])/626)
    class_id.append(class_dict[row["class"]])
    image_id.append(int(row["filename"][0:-4]))
    centerX.append((row["xmin"] + row["xmax"])/2)
    centerY.append((row["ymin"] + row["ymax"])/2)


In [14]:
df_val.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,61.png,1622,626,cam_re,1156,291,1202,329
1,245.png,1622,626,cam_dung_va_do,842,369,850,378
2,245.png,1622,626,cam_con_lai,860,373,868,381
3,261.png,1622,626,cam_re,1156,64,1230,127
4,263.png,1622,626,nguy_hiem,1176,402,1202,422


In [15]:
df_val["centerX"] = np.array(centerX)/1622
df_val["centerY"] = np.array(centerY)/626
df_val["class"] = class_id
df_val['image_id'] = image_id
df_val["width"] = widthnew
df_val["height"] = heightnew

In [16]:
df_val.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,centerX,centerY,image_id
0,61.png,0.028360,0.060703,3,1156,291,1202,329,0.726880,0.495208,61
1,245.png,0.004932,0.014377,2,842,369,850,378,0.521578,0.596645,245
2,245.png,0.004932,0.012780,5,860,373,868,381,0.532676,0.602236,245
3,261.png,0.045623,0.100639,3,1156,64,1230,127,0.735512,0.152556,261
4,263.png,0.016030,0.031949,6,1176,402,1202,422,0.733046,0.658147,263


In [17]:
len(df_val)

776

In [18]:
train_id = [x.strip() for x in train_image_id_list]
len(train_id)

3934

In [19]:
centerX = []
centerY = []
image_id = []
class_id = []
widthnew = []
heightnew = []

for key,row in df_train.iterrows():
    widthnew.append((row["xmax"]-row["xmin"])/1622)
    heightnew.append((row["ymax"]-row["ymin"])/626)
    class_id.append(class_dict[row["class"]])
    image_id.append(int(row["filename"][0:-4]))
    centerX.append((row["xmin"] + row["xmax"])/2)
    centerY.append((row["ymin"] + row["ymax"])/2)

In [20]:
df_train["centerX"] = np.array(centerX)/1622
df_train["centerY"] = np.array(centerY)/626
df_train["class"] = class_id
df_train['image_id'] = image_id
df_train["width"] = widthnew
df_train["height"] = heightnew

In [21]:
df_train.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,area,centerX,centerY,image_id
0,3.png,0.011714,0.028754,2,880,333,899,351,342,0.548397,0.546326,3
1,5.png,0.051171,0.132588,3,1069,355,1152,438,6889,0.684649,0.633387,5
2,5.png,0.005549,0.011182,2,768,480,777,487,63,0.476264,0.772364,5
3,6.png,0.010481,0.023962,6,781,337,798,352,255,0.486745,0.550319,6
4,16.png,0.004316,0.012780,2,733,352,740,360,56,0.454069,0.568690,16


In [22]:
df_train["class"] = df_train["class"] -1

In [23]:
df_train.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,area,centerX,centerY,image_id
0,3.png,0.011714,0.028754,1,880,333,899,351,342,0.548397,0.546326,3
1,5.png,0.051171,0.132588,2,1069,355,1152,438,6889,0.684649,0.633387,5
2,5.png,0.005549,0.011182,1,768,480,777,487,63,0.476264,0.772364,5
3,6.png,0.010481,0.023962,5,781,337,798,352,255,0.486745,0.550319,6
4,16.png,0.004316,0.012780,1,733,352,740,360,56,0.454069,0.568690,16


In [24]:
df_val["class"] = df_val["class"] -1
df_val.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,centerX,centerY,image_id
0,61.png,0.028360,0.060703,2,1156,291,1202,329,0.726880,0.495208,61
1,245.png,0.004932,0.014377,1,842,369,850,378,0.521578,0.596645,245
2,245.png,0.004932,0.012780,4,860,373,868,381,0.532676,0.602236,245
3,261.png,0.045623,0.100639,2,1156,64,1230,127,0.735512,0.152556,261
4,263.png,0.016030,0.031949,5,1176,402,1202,422,0.733046,0.658147,263




# **Extract train_id**





In [25]:
df_train1 = pd.read_csv("/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/annotations/process_csv/train.csv")
len(df_train1)

10001

In [26]:
df_train1.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,3.png,1622,626,cam_dung_va_do,880,333,899,351
1,5.png,1622,626,cam_re,1069,355,1152,438
2,5.png,1622,626,cam_dung_va_do,768,480,777,487
3,6.png,1622,626,nguy_hiem,781,337,798,352
4,16.png,1622,626,cam_dung_va_do,733,352,740,360


In [27]:
centerX = []
centerY = []
image_id = []
class_id = []
widthnew = []
heightnew = []

for key,row in df_train1.iterrows():
    widthnew.append((row["xmax"]-row["xmin"])/1622)
    heightnew.append((row["ymax"]-row["ymin"])/626)
    class_id.append(class_dict[row["class"]])
    image_id.append(int(row["filename"][0:-4]))
    centerX.append((row["xmin"] + row["xmax"])/2)
    centerY.append((row["ymin"] + row["ymax"])/2)

In [28]:
df_train1["centerX"] = np.array(centerX)/1622
df_train1["centerY"] = np.array(centerY)/626
df_train1["class"] = class_id
df_train1['image_id'] = image_id
df_train1["width"] = widthnew
df_train1["height"] = heightnew

In [29]:
df_train1.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,centerX,centerY,image_id
0,3.png,0.011714,0.028754,2,880,333,899,351,0.548397,0.546326,3
1,5.png,0.051171,0.132588,3,1069,355,1152,438,0.684649,0.633387,5
2,5.png,0.005549,0.011182,2,768,480,777,487,0.476264,0.772364,5
3,6.png,0.010481,0.023962,6,781,337,798,352,0.486745,0.550319,6
4,16.png,0.004316,0.012780,2,733,352,740,360,0.454069,0.568690,16


In [30]:
len(df_train1)

10001

In [31]:
train_id = set(df_train1["image_id"])

In [32]:
id_filter = set(df_train["image_id"])

In [33]:
lost_train_id = train_id - id_filter
train_norm = id_filter

In [34]:
print(len(lost_train_id))
# train_norm are list of image id that have at least one box
print(len(train_norm))

140
3934


In [ ]:
lost_train_id

# **Extract Val_id**

In [ ]:
df_val1 = pd.read_csv("/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/annotations/process_csv/validate.csv")
len(df_val1)

999

In [ ]:
df_val1.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,61.png,1622,626,cam_re,1156,291,1202,329
1,245.png,1622,626,cam_dung_va_do,842,369,850,378
2,245.png,1622,626,cam_con_lai,860,373,868,381
3,261.png,1622,626,cam_re,1156,64,1230,127
4,263.png,1622,626,nguy_hiem,1176,402,1202,422


In [ ]:
centerX = []
centerY = []
image_id = []
class_id = []
widthnew = []
heightnew = []

for key,row in df_val1.iterrows():
    widthnew.append((row["xmax"]-row["xmin"])/1622)
    heightnew.append((row["ymax"]-row["ymin"])/626)
    class_id.append(class_dict[row["class"]])
    image_id.append(int(row["filename"][0:-4]))
    centerX.append((row["xmin"] + row["xmax"])/2)
    centerY.append((row["ymin"] + row["ymax"])/2)

In [ ]:
df_val1["centerX"] = np.array(centerX)/1622
df_val1["centerY"] = np.array(centerY)/626
df_val1["class"] = class_id
df_val1['image_id'] = image_id
df_val1["width"] = widthnew
df_val1["height"] = heightnew

In [ ]:
df_val1.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,centerX,centerY,image_id
0,61.png,0.028360,0.060703,3,1156,291,1202,329,0.726880,0.495208,61
1,245.png,0.004932,0.014377,2,842,369,850,378,0.521578,0.596645,245
2,245.png,0.004932,0.012780,5,860,373,868,381,0.532676,0.602236,245
3,261.png,0.045623,0.100639,3,1156,64,1230,127,0.735512,0.152556,261
4,263.png,0.016030,0.031949,6,1176,402,1202,422,0.733046,0.658147,263


In [ ]:
val_id = set(df_val1["image_id"])

In [ ]:
df_val.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,centerX,centerY,image_id
0,61.png,0.028360,0.060703,2,1156,291,1202,329,0.726880,0.495208,61
1,245.png,0.004932,0.014377,1,842,369,850,378,0.521578,0.596645,245
2,245.png,0.004932,0.012780,4,860,373,868,381,0.532676,0.602236,245
3,261.png,0.045623,0.100639,2,1156,64,1230,127,0.735512,0.152556,261
4,263.png,0.016030,0.031949,5,1176,402,1202,422,0.733046,0.658147,263


In [ ]:
len(df_val)

776

In [ ]:
val_id_filter  = set(df_val["image_id"])

In [ ]:
lost_val_id = val_id - val_id_filter
# train_norm are list of image id that have at least one box
val_norm = set(df_val["image_id"])

In [ ]:
print(len(lost_val_id))
# train_norm are list of image id that have at least one box
print(len(val_norm))

14
412


# **Write DataFrame to required format by yolov5**

In [ ]:
%cd /content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/split_data

/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/split_data


In [ ]:
ls

test/  train/


In [ ]:
preid = 0 # dummy image id for initialisation
count = 0
for index, row in df_train.iterrows():
    # write to exist file
    if row["image_id"] == preid:
        path = "train/" + str(row["image_id"]) + ".txt"
        f = open(path, "a")
        # class xcenter ycenter width height
        line = "\n" + str(row["class"]) + " " + str(row["centerX"]) + " "+ str(row["centerY"]) + " "+ str(row["width"]) + " "+ str(row["height"])
        f.write(line)
        f.close()
    
    # create a new file to write
    else:
        path = "train/" + str(row["image_id"]) + ".txt"
        f = open(path, "w")
        # class xcenter ycenter width height
        line = str(row["class"]) + " " + str(row["centerX"]) + " "+ str(row["centerY"]) + " "+ str(row["width"]) + " "+ str(row["height"])
        f.write(line)
        f.close()


    preid = row["image_id"]

In [ ]:
%cd /content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/split_data

/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/split_data


In [ ]:
preid = 0 # dummy image id for initialisation
count = 0
for index, row in df_val.iterrows():
    # write to exist file
    if row["image_id"] == preid:
        if row["image_id"] in val_norm:
            path = "test/" + str(row["image_id"]) + ".txt"
            f = open(path, "a")
        # class xcenter ycenter width height
        line = "\n" + str(row["class"]) + " " + str(row["centerX"]) + " "+ str(row["centerY"]) + " "+ str(row["width"]) + " "+ str(row["height"])
        f.write(line)
        f.close()
    
    # create a new file to write
    else:
        if row["image_id"] in val_norm:
            path = "test/" + str(row["image_id"]) + ".txt"
            f = open(path, "w")
        if row["image_id"] in train_norm:
            path = "train/" + str(row["image_id"]) + ".txt"
            f = open(path, "w")
        # class xcenter ycenter width height
        line = str(row["class"]) + " " + str(row["centerX"]) + " "+ str(row["centerY"]) + " "+ str(row["width"]) + " "+ str(row["height"])
        f.write(line)
        f.close()


    preid = row["image_id"]

In [ ]:
for image_id in lost_train_id:
    path = "/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/split_data/train/" + str(image_id) + ".txt"
    f = open(path, "w")
    f.close()

for image_id in lost_val_id:
    path = "/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/split_data/test/" + str(image_id) + ".txt"
    f = open(path, "w")
    f.close()

In [ ]:
print(len(os.listdir("/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/split_data/train")))

8560


In [ ]:
print(len(os.listdir("/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/split_data/test")))

852


In [ ]:
ls

test/  train/


In [ ]:
raw_data_path = '/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/images'
train_data_path = '/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/split_data/train'
valid_data_path = '/content/drive/MyDrive/TrafficSignDetection/Dataset/traffic_train/split_data/test'


valid_total_files = glob.glob(valid_data_path + "/*.txt")
print("ValidTotal file = ", len(valid_total_files))

train_total_files = glob.glob(train_data_path + "/*.txt")
print("Train Total file = ", len(train_total_files))


Total file =  426
Total file =  4486


In [ ]:
valid_total_files = glob.glob(valid_data_path + "/*.png")
print("ValidTotal file = ", len(valid_total_files))

train_total_files = glob.glob(train_data_path + "/*.png")
print("Train Total file = ", len(train_total_files))

ValidTotal file =  426
Train Total file =  4074
